In [1]:
import torch
from torchvision import models

In [2]:
resnet = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)

preprocess = models.ResNet101_Weights.DEFAULT.transforms()

categories = models.ResNet101_Weights.DEFAULT.meta["categories"]

In [3]:
from PIL import Image
owl = Image.open("../../data/owl.JPG")
owl_t = preprocess(owl)
print("before unsqueeze", owl_t.shape)
batch_t = torch.unsqueeze(owl_t, dim=0)
print("after unsqueeze:", batch_t.shape)

before unsqueeze torch.Size([3, 224, 224])
after unsqueeze: torch.Size([1, 3, 224, 224])


In [4]:
resnet.eval()
with torch.no_grad():
    output = resnet(batch_t)

output.shape

torch.Size([1, 1000])

In [5]:
proba = output[0]
pred = torch.argmax(output[0]).item()
print(categories[pred])
print("accuracy: ", proba[pred].item())

great grey owl
accuracy:  7.81650972366333
